# import related package and read file

In [39]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from sklearn.multiclass import OneVsRestClassifier
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PolynomialDecay
import pandas as pd
import numpy as np

In [2]:
# read in the dataset
train = pd.read_json("train.json")
test = pd.read_json("test.json")

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# feature engineering

## author and venue - One hot

In [4]:
# doing the one-hot for the author and save it in the label.npy for later use
def save_label(train):
    labels = []
    for index, row in train.iterrows():
        au = np.array(row['authors'])
        pa = au[au<100]
        l_a = np.zeros(101)
        if len(pa) == 0:
            l_a[-1] = 1.
        else:
            l_a[pa] = 1.
        labels.append(l_a)
    np.array(labels)[0]
    np.save('labels.npy',labels)
    return

In [5]:
def save_venue(train,file):
    # doing one-hot for the venue 
    venue = []
    for index, row in train.iterrows():
        vn = np.zeros(466)
        v = row["venue"]
        if v == '':
            vn[-1] = 1
        else:
            vn[v] = 1
        venue.append(vn)
    venue = np.array(venue)
    np.save(f'venue_{file}.npy',venue)
    return

In [6]:
  # doing one-hot for the test coauthor 
def test_save_coauthor(test,file):
    coauthor = []
    for index, row in test.iterrows():
        co_vec= np.zeros(22146)
        au = np.array(row['coauthors'])
        pa = au[au<100]
        ca = au[au >=100]
        if len(ca) == 0:
            co_vec[-1] = 1 
        for i in row['coauthors']:
            if i > 99:
                co_vec[i] = 1
        coauthor.append(co_vec[100:])
    coauthor = np.array(coauthor)
    np.save(f'coauthor_{file}.npy',coauthor)
    return

In [7]:
# doing one-hot for the train coauthor 
def train_save_coauthor(train,file):
    coauthor = []
    for index, row in train.iterrows():
        co_vec= np.zeros(22146)
        au = np.array(row['authors'])
        pa = au[au<100]
        ca = au[au >=100]
        if len(ca) == 0:
            co_vec[-1] = 1 
        for i in row['authors']:
            if i > 99:
                co_vec[i] = 1
        coauthor.append(co_vec[100:])
    coauthor = np.array(coauthor)
    np.save(f'coauthor_{file}.npy',coauthor)
    return

In [8]:
save_label(train)

In [9]:
save_venue(train,'train')
save_venue(test,'test')

In [10]:
test_save_coauthor(test,'test')
train_save_coauthor(train,'train')

## Abstract - word2Vector

In [11]:
from gensim.models import Word2Vec

In [12]:
#!pip install --upgrade gensim

In [13]:
vec_size = 1

In [14]:
# save the abstract inforamtion into new array
abstr = []
for index in train["abstract"]:
    abstr.append(index)

In [15]:
# construct the word2vec model
model = Word2Vec(sentences=abstr, vector_size=vec_size, min_count=1)

In [16]:
# train the model 
model.train(abstr,total_examples=model.corpus_count,epochs=5)

(15972681, 20846410)

In [17]:
# help function to get the result of word2vec
def get_vect(abstract, words,wv):
    temp = []
    for index in abstract:
        if index in words:
            temp.append(np.array(wv[index]))
    return np.array(temp)

In [18]:
# get the word list
words = set(model.wv.index_to_key)

In [19]:
# get train vect
array_train = []
for i in range(len(train['abstract'])):
    array_train.append(get_vect(train['abstract'][i], words,model.wv))

train['Train_vect'] = array_train
train

,authors,year,abstract,venue,title,Train_vect
0,"[42, 13720, 36]",9,"[2455, 1858, 2335, 1543, 1800, 1860, 2000, 286...",20,"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1...","[[2.2116356], [1.8893776], [2.474238], [1.9152..."
1,"[1359, 15881, 45]",15,"[40, 1542, 1691, 2449, 1535, 3616, 2206, 1904,...",2,"[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3...","[[1.2804638], [2.0873275], [2.2989619], [1.844..."
2,"[19166, 17763]",17,"[40, 1542, 1691, 2449, 1535, 2610, 1543, 1535,...",,"[2085, 1719, 1846, 1745, 2243, 1553, 1606, 159...","[[1.2804638], [2.0873275], [2.2989619], [1.844..."
3,[97],10,"[46, 1624, 1547, 56, 1687, 1644, 6, 7, 3386, 1...",4,"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5...","[[2.1707966], [2.5829632], [0.9455828], [2.033..."
4,"[19617, 2]",10,"[37, 3709, 3836, 1586, 2151, 1727, 3021, 1860,...",9,"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,...","[[1.7944051], [2.4077144], [1.5956217], [1.979..."
...,...,...,...,...,...,...
25788,"[1797, 78]",14,"[46, 1605, 1681, 10, 1557, 4741, 1535, 2021, 1...",1,"[46, 1910, 36, 2107, 1547, 1553, 1716, 1528, 5...","[[2.1707966], [2.2951586], [2.0774086], [2.025..."
25789,"[18357, 11563, 7905]",11,"[46, 1605, 1691, 10, 2551, 4168, 2030, 3134, 1...",278,"[47, 1574, 1541, 1549, 47, 1966, 2114, 112, 12...","[[2.1707966], [2.2951586], [2.2989619], [2.025..."
25790,"[2236, 20436, 10501]",7,"[1611, 1543, 1535, 2817, 2073, 1647, 11, 2933,...",17,"[56, 1718, 3012, 56, 1902, 2160, 1546, 1623, 1...","[[1.940961], [1.9152185], [1.8496282], [2.0085..."
25791,"[3921, 8760]",16,"[37, 1662, 33, 2007, 1669, 4981, 1650, 1527, 1...",0,"[3066, 1728, 11, 1546, 11, 3066, 1728, 1531, 1...","[[1.7944051], [2.4019234], [1.9825178], [2.904..."


In [20]:
# get test vect
array_test = []
for i in range(len(test['abstract'])):
    array_test.append(get_vect(test['abstract'][i], words,model.wv))

test['Test_vect'] = array_test
test

,identifier,coauthors,year,abstract,venue,title,Test_vect
0,0,"[16336, 1762, 4357, 12564]",19,"[37, 1662, 3207, 10, 33, 2037, 1738, 1642, 155...",223,"[3207, 24, 1798, 1738, 37, 2375, 1568, 11, 53,...","[[1.7944051], [2.4019234], [1.5063787], [2.025..."
1,1,"[21189, 14088]",19,"[1731, 2130, 3674, 1705, 1656, 3077, 1546, 367...",223,"[40, 1560, 1536, 1544, 1609, 1705, 1658, 1543,...","[[1.394289], [2.3695982], [3.2447968], [1.0766..."
2,2,"[3625, 1198, 19889, 794, 2749, 7801]",19,"[1551, 1728, 3920, 1542, 1535, 1656, 1543, 153...",7,"[47, 1574, 1729, 1641, 11, 37, 2533, 2015, 47,...","[[1.4518455], [1.4836818], [2.470573], [2.0873..."
3,3,"[19810, 15173, 5876, 111]",19,"[51, 1535, 2115, 1543, 1811, 1700, 1657, 1684,...",21,"[1770, 53, 2054, 1549, 1529, 1723, 2796, 1547,...","[[1.6170428], [1.8496282], [2.2390532], [1.915..."
4,4,"[10932, 7668, 11907, 19601, 15307, 10492, 1049...",19,"[1775, 1746, 1842, 1525, 33, 2551, 1882, 1542,...",,"[18, 1924, 23, 1544, 3927, 2686, 1543, 1535, 1...","[[1.3783351], [1.1997148], [2.2842307], [1.868..."
...,...,...,...,...,...,...,...
795,795,"[2964, 9493]",19,"[1657, 1533, 1681, 1527, 1667, 2295, 1916, 236...",223,"[2796, 1547, 1531, 1550, 2370, 51, 1549, 2600,...","[[2.4005497], [1.9115583], [2.0774086], [2.432..."
796,796,"[6305, 4866, 5741, 15334]",19,"[1618, 1552, 1630, 2367, 1613, 3600, 1857, 155...",0,"[37, 1747, 2188, 11, 3623, 57, 2103, 1555, 37,...","[[1.6015205], [2.3229764], [2.3714106], [2.599..."
797,797,"[4856, 5063]",19,"[37, 1662, 33, 2095, 1726, 2090, 1553, 2410, 2...",0,"[1621, 11, 1560, 1816, 2151, 1533, 1534, 37, 1...","[[1.7944051], [2.4019234], [1.9825178], [2.309..."
798,798,"[18095, 2884, 15373]",19,"[46, 2337, 11, 2116, 2185, 1839, 3154, 3057, 1...",460,"[1832, 1552, 10, 3585, 10, 35, 1889, 1597, 425...","[[2.1707966], [2.0321214], [2.322046], [2.0399..."


In [21]:
def get_section(array1):
    return list(array1.loc[0:0])

In [22]:
train_Text_vect = get_section(train['Train_vect'])
test_Text_vect = get_section(test['Test_vect'])
#train_Text_vect
#test_Text_vect

In [23]:
def text_avg(data):
    saving = []
    for v in data:
        if v.size:
            saving.append(v.mean(axis=0))
        else:
            saving.append(np.zeros(vec_size, dtype=float))
    return saving

In [24]:
text_vect_avg_train = text_avg(train['Train_vect'])
text_vect_avg_test = text_avg(test['Test_vect'])
train['Train_vect_avg'] = text_vect_avg_train
test['Test_vect_avg'] = text_vect_avg_test

In [25]:
#save the train abstract
df_Machine_Learning = pd.DataFrame(text_vect_avg_train)
df_Machine_Learning.to_csv('w2v_abstract_train.csv')
df_Machine_Learning

,0
0,2.053521
1,2.014920
2,1.977276
3,1.957076
4,1.992952
...,...
25788,2.045703
25789,1.863224
25790,2.007826
25791,2.074274


In [26]:
# save the test abstract
df_Machine_Learning2 = pd.DataFrame(text_vect_avg_test)
df_Machine_Learning2.to_csv('w2v_abstract_test.csv')
df_Machine_Learning2

,0
0,1.988206
1,2.022378
2,1.987639
3,2.011484
4,2.064163
...,...
795,1.987117
796,2.012513
797,1.999069
798,2.098467


## Title - word2Vector

In [27]:
# save the abstract inforamtion into new array
title = []
for d in train["title"]:
    title.append(d)

In [28]:
# construct the word2vec model
model2 = Word2Vec(sentences=title, vector_size=vec_size, min_count=1)

In [29]:
# train the model 
model2.train(title,total_examples=model2.corpus_count,epochs=5)

(1837182, 2608500)

In [30]:
# get the word list
words_title = set(model2.wv.index_to_key)

In [31]:
# get train vect
array_train_title = []
for i in range(len(train['title'])):
    array_train_title.append(get_vect(train['title'][i], words_title,model2.wv))

train['Train_vect_title'] = array_train_title
train

,authors,year,abstract,venue,title,Train_vect,Train_vect_avg,Train_vect_title
0,"[42, 13720, 36]",9,"[2455, 1858, 2335, 1543, 1800, 1860, 2000, 286...",20,"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1...","[[2.2116356], [1.8893776], [2.474238], [1.9152...",[2.0535214],"[[2.792205], [3.0835714], [2.3256614], [3.3058..."
1,"[1359, 15881, 45]",15,"[40, 1542, 1691, 2449, 1535, 3616, 2206, 1904,...",2,"[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3...","[[1.2804638], [2.0873275], [2.2989619], [1.844...",[2.0149202],"[[2.6105237], [2.4216251], [2.3784254], [2.968..."
2,"[19166, 17763]",17,"[40, 1542, 1691, 2449, 1535, 2610, 1543, 1535,...",,"[2085, 1719, 1846, 1745, 2243, 1553, 1606, 159...","[[1.2804638], [2.0873275], [2.2989619], [1.844...",[1.9772763],"[[2.2408094], [1.8865498], [3.3643222], [2.481..."
3,[97],10,"[46, 1624, 1547, 56, 1687, 1644, 6, 7, 3386, 1...",4,"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5...","[[2.1707966], [2.5829632], [0.9455828], [2.033...",[1.9570763],"[[2.1650357], [6.027611], [1.9367796], [1.7209..."
4,"[19617, 2]",10,"[37, 3709, 3836, 1586, 2151, 1727, 3021, 1860,...",9,"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,...","[[1.7944051], [2.4077144], [1.5956217], [1.979...",[1.9929516],"[[1.5250256], [4.5536723], [2.0059564], [2.167..."
...,...,...,...,...,...,...,...,...
25788,"[1797, 78]",14,"[46, 1605, 1681, 10, 1557, 4741, 1535, 2021, 1...",1,"[46, 1910, 36, 2107, 1547, 1553, 1716, 1528, 5...","[[2.1707966], [2.2951586], [2.0774086], [2.025...",[2.0457034],"[[2.392672], [3.5318594], [2.4218838], [2.3148..."
25789,"[18357, 11563, 7905]",11,"[46, 1605, 1691, 10, 2551, 4168, 2030, 3134, 1...",278,"[47, 1574, 1541, 1549, 47, 1966, 2114, 112, 12...","[[2.1707966], [2.2951586], [2.2989619], [2.025...",[1.8632241],"[[2.4216251], [2.7711713], [2.6365602], [2.102..."
25790,"[2236, 20436, 10501]",7,"[1611, 1543, 1535, 2817, 2073, 1647, 11, 2933,...",17,"[56, 1718, 3012, 56, 1902, 2160, 1546, 1623, 1...","[[1.940961], [1.9152185], [1.8496282], [2.0085...",[2.0078263],"[[2.4676914], [3.6362076], [1.9821275], [2.467..."
25791,"[3921, 8760]",16,"[37, 1662, 33, 2007, 1669, 4981, 1650, 1527, 1...",0,"[3066, 1728, 11, 1546, 11, 3066, 1728, 1531, 1...","[[1.7944051], [2.4019234], [1.9825178], [2.904...",[2.074274],"[[3.6592839], [1.8101674], [2.3784254], [2.231..."


In [32]:
# get test vect
array_test_title = []
for i in range(len(test['title'])):
    array_test_title.append(get_vect(test['title'][i], words_title,model2.wv))

test['Test_vect_title'] = array_test_title
test

,identifier,coauthors,year,abstract,venue,title,Test_vect,Test_vect_avg,Test_vect_title
0,0,"[16336, 1762, 4357, 12564]",19,"[37, 1662, 3207, 10, 33, 2037, 1738, 1642, 155...",223,"[3207, 24, 1798, 1738, 37, 2375, 1568, 11, 53,...","[[1.7944051], [2.4019234], [1.5063787], [2.025...",[1.9882061],"[[2.9717996], [2.1912708], [3.387935], [2.5878..."
1,1,"[21189, 14088]",19,"[1731, 2130, 3674, 1705, 1656, 3077, 1546, 367...",223,"[40, 1560, 1536, 1544, 1609, 1705, 1658, 1543,...","[[1.394289], [2.3695982], [3.2447968], [1.0766...",[2.022378],"[[2.1650357], [2.2548304], [4.093365], [2.2072..."
2,2,"[3625, 1198, 19889, 794, 2749, 7801]",19,"[1551, 1728, 3920, 1542, 1535, 1656, 1543, 153...",7,"[47, 1574, 1729, 1641, 11, 37, 2533, 2015, 47,...","[[1.4518455], [1.4836818], [2.470573], [2.0873...",[1.9876388],"[[2.4216251], [2.7711713], [2.6967318], [3.306..."
3,3,"[19810, 15173, 5876, 111]",19,"[51, 1535, 2115, 1543, 1811, 1700, 1657, 1684,...",21,"[1770, 53, 2054, 1549, 1529, 1723, 2796, 1547,...","[[1.6170428], [1.8496282], [2.2390532], [1.915...",[2.0114844],"[[3.1148438], [2.6438768], [3.6261728], [2.102..."
4,4,"[10932, 7668, 11907, 19601, 15307, 10492, 1049...",19,"[1775, 1746, 1842, 1525, 33, 2551, 1882, 1542,...",,"[18, 1924, 23, 1544, 3927, 2686, 1543, 1535, 1...","[[1.3783351], [1.1997148], [2.2842307], [1.868...",[2.064163],"[[2.0720525], [3.031718], [2.2072327], [2.2177..."
...,...,...,...,...,...,...,...,...,...
795,795,"[2964, 9493]",19,"[1657, 1533, 1681, 1527, 1667, 2295, 1916, 236...",223,"[2796, 1547, 1531, 1550, 2370, 51, 1549, 2600,...","[[2.4005497], [1.9115583], [2.0774086], [2.432...",[1.9871166],"[[3.2921739], [2.5395792], [2.7418532], [3.083..."
796,796,"[6305, 4866, 5741, 15334]",19,"[1618, 1552, 1630, 2367, 1613, 3600, 1857, 155...",0,"[37, 1747, 2188, 11, 3623, 57, 2103, 1555, 37,...","[[1.6015205], [2.3229764], [2.3714106], [2.599...",[2.0125132],"[[2.4900973], [1.510247], [2.5779083], [2.3784..."
797,797,"[4856, 5063]",19,"[37, 1662, 33, 2095, 1726, 2090, 1553, 2410, 2...",0,"[1621, 11, 1560, 1816, 2151, 1533, 1534, 37, 1...","[[1.7944051], [2.4019234], [1.9825178], [2.309...",[1.9990693],"[[2.9602158], [2.3784254], [2.2548304], [3.318..."
798,798,"[18095, 2884, 15373]",19,"[46, 2337, 11, 2116, 2185, 1839, 3154, 3057, 1...",460,"[1832, 1552, 10, 3585, 10, 35, 1889, 1597, 425...","[[2.1707966], [2.0321214], [2.322046], [2.0399...",[2.0984666],"[[4.371261], [2.829147], [2.2526493], [3.02267..."


In [33]:
# get the text position
train_Text_vect = get_section(train['Train_vect_title'])
test_Text_vect = get_section(test['Test_vect_title'])

In [34]:
# output the result base on the word set
text_vect_avg_train_title = text_avg(train['Train_vect_title'])
text_vect_avg_test_title = text_avg(test['Test_vect_title'])
train['Train_vect_title_avg'] = text_vect_avg_train_title
test['Test_vect_title_avg'] = text_vect_avg_test_title

In [35]:
# save train title
df_Machine_Learning_title = pd.DataFrame(text_vect_avg_train_title)
df_Machine_Learning_title.to_csv('w2v_title_train.csv')
df_Machine_Learning_title

,0
0,2.630859
1,2.542738
2,2.265837
3,2.801138
4,3.110825
...,...
25788,3.343459
25789,2.392321
25790,2.496149
25791,2.712478


In [36]:
# save test title
df_Machine_Learning_title2 = pd.DataFrame(text_vect_avg_test_title)
df_Machine_Learning_title2.to_csv('w2v_title_test.csv')
df_Machine_Learning_title2

,0
0,3.014898
1,2.539772
2,2.674256
3,2.483784
4,2.459538
...,...
795,2.631144
796,2.508835
797,2.754486
798,2.849071


# Import and concate processed data 

In [37]:
# lables
y = np.load('labels.npy')

In [38]:
# load dataset
x = np.load('venue_train.npy')
coaur = np.load('coauthor_train.npy')
print(x.shape)

(25793, 466)


In [40]:
# train data
x = np.concatenate((x,coaur), axis=1)
abst = np.array(pd.read_csv('w2v_abstract_train.csv'))
x = np.concatenate((x,abst), axis=1)

In [41]:
# normalize
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [42]:
#test data
test_data = np.load('venue_test.npy')
coaur = np.load('coauthor_test.npy')
test_data = np.concatenate((test_data,coaur), axis=1)
abst = np.array(pd.read_csv('w2v_abstract_test.csv'))
test_data = np.concatenate((test_data,abst), axis=1)

In [43]:
test_data = scaler.transform(test_data)

# Navie Bayes model

In [46]:
# from sklearn.naive_bayes import GaussianNB
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
# Gau_NB = GaussianNB()
# clf=OneVsRestClassifier(estimator=Gau_NB,n_jobs=2)
# clf.fit(X_train, y_train)
# y_predict = clf.predict(X_test)
# f1_score(y_test, y_predict, average='sample')
# output_result = clf.predict(test_data)

# Deep learning model 

In [47]:
# create deep learning model
def deep_model(x,y,lr,e,drop,batch,patience):
    model = Sequential()
    model.add(Dense(2048,input_dim = x.shape[1],activation='relu'))
    model.add(Dropout(drop))
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(drop))
    model.add(Dense(512,activation='tanh'))
    model.add(Dropout(drop))
    model.add(Dense(256,activation='tanh'))
    model.add(Dropout(drop))
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(drop))
    model.add(Dense(y.shape[1],activation='sigmoid'))
    lr_schedule = PolynomialDecay(initial_learning_rate = lr,decay_steps=150,end_learning_rate=0.00025,power=0.5)
    model.compile(loss='binary_crossentropy',optimizer=Adam(lr_schedule))
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience,restore_best_weights=True)
    model.fit(x,y,epochs=e,batch_size=batch,callbacks=[callback]) ## if use alternative2 add ,class_weight=class_W
    return model

In [48]:
# set the input parameter according to your demand
lr=0.001
epoch=100
dropout=0.3
batch=64
patience=10

In [ ]:
deep_model = deep_model(x,y,lr,epoch,dropout,batch,patience)

In [ ]:
file_name = f'{lr}_{patience}_{epoch}_{dropout}_{batch}_{v_size}_model'

In [ ]:
pred = deep_model.predict(x)
pred = np.where(pred>=0.5,1,0)
f1_score(y_true = y,y_pred=pred,average='samples')

In [ ]:
prediction = deep_model.predict(test_data)
prediction = np.where(prediction >= 0.5, 1, 0)
idx = [np.where(p >= 0.5) for p in prediction]

In [ ]:
predict = pd.DataFrame(idx, columns=['Predict'])
for i, row in predict.iterrows():
    if len(row['Predict']) > 10:
        row['Predict'] = -1
predict.to_csv(f'{file_name}.csv')